In [14]:
import pickle
import numpy as np
from scipy.spatial import distance

In [3]:
# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [6]:
#create dictionary with words a labels and the EEG embeddings in a list as the values

EEG_word_level_dict = {}
for i in range(len(EEG_word_level_labels)):
    if EEG_word_level_labels[i] in EEG_word_level_dict:
        EEG_word_level_dict[EEG_word_level_labels[i]].append(EEG_word_level_embeddings[i])
    else:
        EEG_word_level_dict[EEG_word_level_labels[i]] = [EEG_word_level_embeddings[i]]


In [15]:
def convert_to_probability_distribution(eeg_segment):
    """
    Convert EEG segment to a probability distribution.

    Parameters:
        eeg_segment (array-like): EEG segment data.

    Returns:
        array-like: Probability distribution representing the EEG segment.
    """
    # Perform feature extraction (e.g., compute spectral power densities)
    # Example: Calculate mean power spectral density across channels
    power_spectral_density = np.mean(np.abs(np.fft.fft(eeg_segment)) ** 2, axis=1)

    # Normalize the extracted features
    normalized_features = (power_spectral_density - np.mean(power_spectral_density)) / np.std(power_spectral_density)

    # Convert normalized features into probability values
    # Example: Apply softmax function
    probabilities = np.exp(normalized_features) / np.sum(np.exp(normalized_features))

    return probabilities